# AB test 

A/B testing is a research method that allows you to find out people's reaction to any changes. The study shows which of the two versions of the product or offer is better and gives greater effect.

In [33]:
import random

from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole
from hypex import ABTest

## Creation of a new test dataset with synthetic data.
It is important to mark the data fields by assigning the appropriate roles:

* FeatureRole: a role for columns that contain features or predictor variables. Our split will be based on them. Applied by default if the role is not specified for the column.
* TreatmentRole: a role for columns that show the treatment or intervention.
* TargetRole: a role for columns that show the target or outcome variable.
* InfoRole: a role for columns that contain information about the data, such as user IDs.

In [34]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(), 
        "gender": TargetRole()
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [35]:
data["treat"] = [random.choice([0, 1, 2]) for _ in range(len(data))]
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      2       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      2       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      2       538.5   450.444444  42.0      M   
9996     9996             0      2       500.5   430.888889  26.0      F   
9997     9997             3      2       473.0   534.111111  22.0      F   
9998     9998             2      0       495.0   523.222222  67.0      F   
9999     9999             7      2       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [36]:
data.roles

{'user_id': Info(<class 'int'>),
 'treat': Treatment(<class 'int'>),
 'pre_spends': Target(<class 'float'>),
 'post_spends': Target(<class 'float'>),
 'gender': Target(<class 'str'>),
 'signup_month': Default(<class 'int'>),
 'age': Default(<class 'float'>),
 'industry': Default(<class 'str'>)}

## Simple AB Test 

Simple pipline contains group sizes, group differences and TTest estimation. 

In [37]:
test = ABTest()
result = test.execute(data)

In [38]:
result.resume

       feature group TTest pass  TTest p-value  control mean   test mean  \
0   pre_spends     1     NOT OK       0.128774     487.39089  487.200741   
1  post_spends     1     NOT OK       0.904016     487.39089  486.687276   

   difference  difference %  
0   -0.190149     -0.039014  
1   -0.703614     -0.144363  

In [39]:
result.sizes 

         control size  test size  control size %  test size % group
1┆treat          3238       3348              49           50     1
2┆treat          3238       3414              48           51     2

In [40]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

## Additional tests in AB Test 

It is possible to add u-test and chi2-test in pipline. 

In [41]:
test = ABTest(additional_tests=['t-test', 'u-test', 'chi2-test'])
result = test.execute(data)

In [42]:
result.resume

       feature group TTest pass  TTest p-value UTest pass  UTest p-value  \
0   pre_spends     1     NOT OK       0.128774     NOT OK       0.299735   
1  post_spends     1     NOT OK       0.904016     NOT OK       0.810312   

  Chi2Test pass  Chi2Test p-value  control mean   test mean  difference  \
0           NaN               NaN     487.39089  487.200741   -0.190149   
1           NaN               NaN     487.39089  486.687276   -0.703614   

   difference %  
0     -0.039014  
1     -0.144363  

In [43]:
result.multitest

"There was less than three groups or multitest method wasn't provided"

In [44]:
result.sizes

          control size  test size  control size %  test size % group
1┆gender          3238       3348              49           50     1
2┆gender          3238       3414              48           51     2

## ABn Test 

Finally, we may estimate multiple ab test with different methods.

In [45]:
test = ABTest(multitest_method="bonferroni")
result = test.execute(data)

In [46]:
result.resume

       feature group TTest pass  TTest p-value  control mean   test mean  \
0   pre_spends     1     NOT OK       0.128774     487.39089  487.200741   
1  post_spends     1     NOT OK       0.904016     487.39089  486.687276   

   difference  difference %  
0   -0.190149     -0.039014  
1   -0.703614     -0.144363  

In [47]:
result.sizes

          control size  test size  control size %  test size % group
1┆gender          3238       3348              49           50     1
2┆gender          3238       3414              48           51     2

In [48]:
result.multitest

   correction        field  new p-value  old p-value  rejected   test group
0    0.680658   pre_spends     1.000000     0.680658     False  TTest     1
1    0.250000  post_spends     0.515098     0.128774     False  TTest     1
2    0.755965   pre_spends     1.000000     0.755965     False  TTest     2
3    0.904016  post_spends     1.000000     0.904016     False  TTest     2